In [ ]:
import torch
print("CUDA available: ", torch.cuda.is_available())
!nvidia-smi

In [ ]:
!pip -q install transformers datasets evaluate accelerate --upgrade

In [ ]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

ckpt = "bert-base-uncased" # base BERT
ds = load_dataset("glue", "sst2") # SST-2 sentiment dataset
tok = AutoTokenizer.from_pretrained(ckpt)

def tokenize(batch):
  return tok(batch["sentence"], truncation=True, max_length=128)

ds_tok = ds.map(tokenize, batched=True, remove_columns=["sentence"])


In [ ]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np, evaluate

collator = DataCollatorWithPadding(tokenizer=tok)
model = AutoModelForSequenceClassification.from_pretrained(ckpt, num_labels=2)

metric = evaluate.load("glue", "sst2")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return metric.compute(predictions=preds, references=labels)
args = TrainingArguments(
    output_dir="bert-sst2-t4",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    do_eval=True,
    save_steps=500,
    eval_steps=500,
    report_to="none",
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

In [ ]:
train_result = trainer.train()
train_result

In [ ]:
metrics = trainer.evaluate()
metrics

In [ ]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=trainer.model, tokenizer=tok, device=0)
print(pipe("This movie was absolutely fantastic!"))
print(pipe("That was the worst meal I've had in years."))

In [ ]:
save_dir = "bert-sst2-t4-best"
trainer.model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)

In [ ]:
!ls /content

In [ ]:
!git config --global user.name llDanieLll
!git config --global user.email danielqiujiahao@gmail.com

# Clone your repo (replace with your repo URL)
!git clone https://github.com/llDanieLll/BERT_FineTuned.git
%cd BERT_FineTuned
!cp /content ./bert_sst2_finetune.ipynb

with open("requirements.txt","w") as f:
  f.write("transformers>=4.44\ndatasets>=2.20\nevaluate>=0.4\naccelerate>=0.33\ntorch\n")

!git add bert_sst2_finetune.ipynb requirements.txt
!git commit -m "Add Colab notebook and requirements"

!git push

In [ ]:
from google.colab import files
files.download("Training.ipynb")